In [69]:
import numpy as np
import os
import matplotlib.pyplot as plt
import librosa
import joblib
from glob import glob

Ler os arquivos de áudio para coletar os coeficientes e os os labels. Os labels serão divididos da seguinte maneira:
- Anger: 0
- Disgust: 1
- Fear: 2
- Happy: 3
- Neutral: 4
- Sad: 5

In [70]:
base_dir = 'data/'

Vou ler apenas 150 audios de cada emoção pq eu não quero queimar meu computador

In [83]:
label_map = {'notScreaming': 0, 'screaming': 1}
lst = []
number_of_audios_per_emotion = -1 # set -1 for all audios

for emotion in label_map.keys():
    folder_path = os.path.join(base_dir, emotion)
    audios_number = 0
    # Iterate over the audio files in the folder
    for audio_file in glob('./data/' + emotion + '/*.wav'):
        # print(os.path.join(folder_path, audio_file))
        sr = 4000
        X, sample_rate = librosa.load(audio_file, sr=sr)
        mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sr, n_mfcc=40).T,axis=0)
        file = label_map[emotion]
        arr = mfccs, file
        lst.append(arr)
        audios_number += 1
        if(audios_number == number_of_audios_per_emotion): break
    print(audios_number, emotion, 'audio files loaded')

/home/gta/chiacchio/ProcVoz/env/lib/python3.10/site-packages/librosa/core/spectrum.py:266: UserWarning: n_fft=2048 is too large for input signal of length=2044
  warnings.warn(


2631 notScreaming audio files loaded
862 screaming audio files loaded


In [101]:
X, y = zip(*lst)

In [102]:
X = np.asarray(X)
y = np.asarray(y)


X.shape, y.shape

((3493, 40), (3493,))

In [103]:
import joblib

X_name = 'X.joblib'
y_name = 'y.joblib'
save_dir = 'data'

savedX = joblib.dump(X, os.path.join(save_dir, X_name))
savedy = joblib.dump(y, os.path.join(save_dir, y_name))

In [104]:
X = joblib.load('data/X.joblib')
y = joblib.load('data/y.joblib')

In [105]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

## Neural Network

In [106]:
x_traincnn = np.expand_dims(X_train, axis=2)
x_testcnn = np.expand_dims(X_test, axis=2)

In [107]:
x_traincnn.shape, x_testcnn.shape

((2340, 40, 1), (1153, 40, 1))

In [108]:
import keras
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.utils import to_categorical
from keras.layers import Input, Flatten, Dropout, Activation
from keras.layers import Conv1D, MaxPooling1D
from keras.models import Model
from keras.callbacks import ModelCheckpoint
from tensorflow.keras import optimizers
# model = Sequential()

# model.add(Conv1D(128, 5,padding='same',  input_shape=(40,1)))
# model.add(Activation('relu'))
# model.add(Dropout(0.1))
# model.add(MaxPooling1D(pool_size=(6)))
# model.add(Conv1D(128, 5,padding='same',))
# model.add(Activation('relu'))
# model.add(Dropout(0.1))
# model.add(Flatten())
# model.add(Dense(6))
# model.add(Activation('softmax'))
# opt = optimizers.RMSprop(learning_rate=0.00005, rho=0.9, epsilon=None, decay=0.0)

## Modelo 4

Acuracia de 0.87

In [109]:
callback = keras.callbacks.EarlyStopping(monitor='loss',patience= 10)
# callbacks = [
#     keras.callbacks.ModelCheckpoint(filepath="model_at_epoch_{epoch}.keras"),
#     keras.callbacks.EarlyStopping(monitor="val_loss", patience=10),
# ]
model = Sequential()
model.add(Conv1D(64, 5, padding='same',
                input_shape=(40, 1)))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(6))
model.add(Activation('softmax'))

model.compile(loss='sparse_categorical_crossentropy',
        optimizer='rmsprop',
        metrics=['accuracy'])

cnn_history = model.fit(x_traincnn, y_train,
                batch_size=16, epochs=1000,
                validation_data=(x_testcnn, y_test), callbacks=[callback])

Epoch 1/1000


147/147 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7214 - loss: 1.1980 - val_accuracy: 0.6982 - val_loss: 0.8178
Epoch 2/1000
147/147 ━━━━━━━━━━━━━━━━━━━━ 0s 887us/step - accuracy: 0.7728 - loss: 0.7683 - val_accuracy: 0.7589 - val_loss: 0.5284
Epoch 3/1000
147/147 ━━━━━━━━━━━━━━━━━━━━ 0s 889us/step - accuracy: 0.7806 - loss: 0.5915 - val_accuracy: 0.6114 - val_loss: 0.9709
Epoch 4/1000
147/147 ━━━━━━━━━━━━━━━━━━━━ 0s 888us/step - accuracy: 0.7939 - loss: 0.5420 - val_accuracy: 0.8101 - val_loss: 0.4728
Epoch 5/1000
147/147 ━━━━━━━━━━━━━━━━━━━━ 0s 895us/step - accuracy: 0.8282 - loss: 0.4361 - val_accuracy: 0.8040 - val_loss: 0.4299
Epoch 6/1000
147/147 ━━━━━━━━━━━━━━━━━━━━ 0s 888us/step - accuracy: 0.8109 - loss: 0.4238 - val_accuracy: 0.8135 - val_loss: 0.5125
Epoch 7/1000
147/147 ━━━━━━━━━━━━━━━━━━━━ 0s 915us/step - accuracy: 0.8183 - loss: 0.4137 - val_accuracy: 0.8153 - val_loss: 0.4410
Epoch 8/1000
147/147 ━━━━━━━━━━━━━━━━━━━━ 0s 895us/step - accuracy: 0.8119 - loss: 0.4159 -

In [110]:
predictions = np.argmax(model.predict(x_testcnn),axis=1)

37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 


In [111]:
from sklearn.metrics import classification_report
report = classification_report(y_test, predictions)
print(report)

              precision    recall  f1-score   support

           0       0.89      0.83      0.86       863
           1       0.58      0.68      0.63       290

    accuracy                           0.80      1153
   macro avg       0.73      0.76      0.74      1153
weighted avg       0.81      0.80      0.80      1153



## Modelo https://www.kaggle.com/code/shivamburnwal/speech-emotion-recognition?scriptVersionId=34958802&cellId=52

Acurácia de 0.79

In [112]:
from keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout, BatchNormalization

In [136]:
model=Sequential()
model.add(Conv1D(256, kernel_size=5, strides=1, padding='same', activation='relu', input_shape=(x_traincnn.shape[1], 1)))
model.add(BatchNormalization())
model.add(MaxPooling1D(pool_size=5, strides = 2, padding = 'same'))

model.add(Conv1D(256, kernel_size=5, strides=1, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=5, strides = 2, padding = 'same'))

model.add(Conv1D(128, kernel_size=5, strides=1, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=5, strides = 2, padding = 'same'))
model.add(Dropout(0.2))

model.add(Conv1D(64, kernel_size=5, strides=1, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=5, strides = 2, padding = 'same'))

model.add(Flatten())
model.add(Dense(units=32, activation='relu'))
model.add(Dropout(0.3))
model.add(BatchNormalization())

model.add(Dense(units=2, activation='softmax'))
model.compile(optimizer = 'adam' , loss = 'categorical_crossentropy' , metrics = ['accuracy'])

model.summary()

Model: "sequential_15"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_32 (Conv1D)              │ (None, 40, 256)        │         1,536 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_11          │ (None, 40, 256)        │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_23 (MaxPooling1D) │ (None, 20, 256)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_33 (Conv1D)              │ (None, 20, 256)        │       327,936 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_24 (MaxPooling1D) │ (None, 10, 256)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_34 (Conv1D)              │ (None, 10, 128)        │       163,968 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_25 (MaxPooling1D) │ (None, 5, 128)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_19 (Dropout)            │ (None, 5, 128)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_35 (Conv1D)              │ (None, 5, 64)          │        41,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_26 (MaxPooling1D) │ (None, 3, 64)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_14 (Flatten)            │ (None, 192)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_19 (Dense)                │ (None, 32)             │         6,176 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_20 (Dropout)            │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_12          │ (None, 32)             │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_20 (Dense)                │ (None, 2)              │            66 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 541,858 (2.07 MB)

 Trainable params: 541,282 (2.06 MB)

 Non-trainable params: 576 (2.25 KB)

In [138]:
from tensorflow.keras.utils import to_categorical

In [137]:
y_train_encoded = to_categorical(y_train, num_classes=2)
y_test_encoded = to_categorical(y_test, num_classes=2)

In [139]:
y_train_encoded.shape

(2340, 2)

In [140]:
callback = keras.callbacks.EarlyStopping(monitor='loss',patience= 10)
history=model.fit(x_traincnn, y_train_encoded, batch_size=60, epochs=100, validation_data=(x_testcnn, y_test_encoded), callbacks=[callback])

Epoch 1/100


39/39 ━━━━━━━━━━━━━━━━━━━━ 2s 22ms/step - accuracy: 0.5838 - loss: 0.7625 - val_accuracy: 0.7485 - val_loss: 1.3348
Epoch 2/100
39/39 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - accuracy: 0.6438 - loss: 0.6203 - val_accuracy: 0.7485 - val_loss: 0.5920
Epoch 3/100
39/39 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - accuracy: 0.7552 - loss: 0.5073 - val_accuracy: 0.7962 - val_loss: 0.4366
Epoch 4/100
39/39 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - accuracy: 0.7997 - loss: 0.4747 - val_accuracy: 0.8213 - val_loss: 0.4097
Epoch 5/100
39/39 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - accuracy: 0.8308 - loss: 0.4083 - val_accuracy: 0.8274 - val_loss: 0.4095
Epoch 6/100
39/39 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - accuracy: 0.8283 - loss: 0.4088 - val_accuracy: 0.8248 - val_loss: 0.3950
Epoch 7/100
39/39 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - accuracy: 0.8394 - loss: 0.3849 - val_accuracy: 0.8205 - val_loss: 0.4077
Epoch 8/100
39/39 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - accuracy: 0.8336 - loss: 0.3863 - val_accuracy: 0.8196 - val_l

In [141]:
predictions = np.argmax(model.predict(x_testcnn),axis=1)

 1/37 ━━━━━━━━━━━━━━━━━━━━ 2s 74ms/step

37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step


In [144]:
predictions_encoded = to_categorical(predictions, num_classes=2)

In [145]:
from sklearn.metrics import classification_report
report = classification_report(y_test_encoded, predictions_encoded)
print(report)

              precision    recall  f1-score   support

           0       0.81      0.95      0.88       863
           1       0.69      0.35      0.47       290

   micro avg       0.80      0.80      0.80      1153
   macro avg       0.75      0.65      0.67      1153
weighted avg       0.78      0.80      0.77      1153
 samples avg       0.80      0.80      0.80      1153



In [146]:
MODEL_DIR_PATH = 'models'

In [147]:
if not os.path.isdir(MODEL_DIR_PATH):
    os.makedirs(MODEL_DIR_PATH)
model_path = os.path.join(MODEL_DIR_PATH, 'Emotion_Voice_Detection_4_layers_cnn.keras')
model.save(model_path)
print('Saved trained model at %s ' % model_path)

Saved trained model at models/Emotion_Voice_Detection_4_layers_cnn.keras 
